### Project Imports

In [2]:
import os.path
import numpy as np
import pandas as pd
import torch
import pickle
import subprocess
from matplotlib import pyplot as plt
from Bio import SeqIO
from importlib import reload

import minimal_version.parser_handler
%cd /Users/dahala/GitHub/VAE-enzymes/vae_notebooks/
from minimal_version.preprocess_msa import Preprocessor, weight_sequences
from minimal_version.msa import MSA
from minimal_version.utils import Capturing, store_to_pkl, store_to_fasta, load_from_pkl

from minimal_version.train import setup_train, Train
%cd /Users/dahala/GitHub/VAE-enzymes/vae_notebooks/notebooks/

/Users/dahala/GitHub/VAE-enzymes/vae_notebooks
/Users/dahala/GitHub/VAE-enzymes/vae_notebooks/notebooks


### Project Configuration
Configure your project using the configuration file. Use the modelConfig template as a guide. 

A value of -1 indicates that the default value will be used. 

Please note that every time the RunSetup class is executed, the current version of the configuration file will be copied to your specified directory, allowing you to review all set parameters at any time.  

Run just one of the two cell below (working with/without Pfam)

In [ ]:
#CONFIGURATION_FILE = "pfamGT1Small_filtered_hmm.json"
#CONFIGURATION_FILE = "msaEnzymeMiner_PtUGT1.json"
CONFIGURATION_FILE = "msaGASP_bigMSA.json"
run = minimal_version.parser_handler.RunSetup(CONFIGURATION_FILE)
print(f" Working with {CONFIGURATION_FILE} configuration file!")
PFAM_INPUT = False

In [ ]:
sub_cids = [7257, 79964, 445154, 5315892, 5280343, 444539, 6549, 445858, 1548943, 932]
sub_idx = 0
#CONFIGURATION_FILE = f"json_files_GASPcondition/msaGASP_bigMSA_{sub_cids[sub_idx]}.json"
CONFIGURATION_FILE = f"json_files_GASPcondition/msaEnzymeMiner_PtUGT1_{sub_cids[sub_idx]}.json"
#CONFIGURATION_FILE = f"json_files_GASPcondition/msaEnzymeMiner_PtUGT1_3dim_{sub_cids[sub_idx]}.json"
run = minimal_version.parser_handler.RunSetup(CONFIGURATION_FILE)
print(f" Working with {CONFIGURATION_FILE} configuration file!")
PFAM_INPUT = False

In [3]:
CONFIGURATION_FILE = f"json_files_GASPcondition/msaEnzymeMiner_PtUGT1_DCA_DCP_345.json"
#CONFIGURATION_FILE = f"json_files_GASPcondition/msaEnzymeMiner_PtUGT1_DCA_DCP_200.json"
run = minimal_version.parser_handler.RunSetup(CONFIGURATION_FILE)
print(f" Working with {CONFIGURATION_FILE} configuration file!")
PFAM_INPUT = False

Configuration file stored in ../results/EnzymeMiner_PtUGT1_DCA_DCP_345/config/2024-09-30_10-42.json
 Working with json_files_GASPcondition/msaEnzymeMiner_PtUGT1_DCA_DCP_345.json configuration file!


In [ ]:
CONFIGURATION_FILE = "pfamGT1Small.json"
run = minimal_version.parser_handler.RunSetup(CONFIGURATION_FILE)
print(f" Working with {CONFIGURATION_FILE} configuration file!")
PFAM_INPUT = True

### HMM alignment preprocessing
This preprocessing step generates a Hidden Markov Model (HMM) profile from a Multiple Sequence Alignment (MSA) and realigns the sequences from the MSA to this profile, resulting in a HMM-MSA. 

The HMM-MSA can then be utilized as a classical input MSA in the Variational Autoencoder (VAE) pipeline, offering the advantage of facilitating the alignment of different query sequences to the HMM profile. If this option is enabled, ensure that the dataset path in the configuration file is updated to the new fixed path to maintain consistency in future experiments.

Install hmmer to your system first: for example **apt install hmmer** 

In [ ]:
# If you want to use hmm aligner run this cell
hmmAligner = HMMAligner(run)
hmmAligner.buildHMM()

In [ ]:
hmmAligner.hmmer_align("yourCustomFasta")  # you can align your custom sequences to the hmm model here 

### MSA preprocessing
Prepare your MSA for training by filtering it according to the queries specified in the configuration file. This includes both the query sequences and any fixed sequences defined.

In [4]:
"""
Preprocess MSA and prepare it for VAE model training
"""

# logging
msa_log = open(os.path.join(run.logs, 'msa_log.txt'), "w")

# MSA loading
if PFAM_INPUT:
    msa = MSA.load_pfam(run.dataset)
else:
    msa = MSA.load_msa(run.dataset)
MSA.amino_acid_dict(run.pickles)
msg = f"MSA in {run.dataset} loaded\n" \
      f"number of sequences: {len(list(msa.keys()))}"
print(msg)
msa_log.write(msg + f'\n{"="*80}\n')

# MSA preprocessing
preprocessor = Preprocessor(run)
with Capturing() as output:
    msa, msa_keys = preprocessor.trim_msa(msa)
msa_log.write(" MSA Preprocessing \n" + "\n".join(output) + f'\n{"="*80}\n')
assert (msa.shape[0] == len(msa_keys))
trim_file_path = os.path.join(run.msa, "trimmed_msa.fasta")
msg = f"Trimmed MSA has {msa.shape[0]} sequences and the width is {msa.shape[1]}\n" \
      f"Trimmed MSA is stored at {trim_file_path}"
print(msg)
msa_log.write(msg + f'\n{"="*80}\n')
msa_num_dict_shuffled = {k: seq for k, seq in zip(msa_keys, msa)}  # transform to dictionary, have keys together
# but secure that query and fixed sequences are at the beginning 
msa_num_dict = {k: msa_num_dict_shuffled[k] for k in preprocessor.keep_keys}
msa_num_dict.update({k: msa_num_dict_shuffled[k] for k in msa_num_dict_shuffled if k not in preprocessor.keep_keys})

trimmed_msa = MSA.number_to_amino(msa_num_dict)
store_to_fasta(trimmed_msa, trim_file_path)
store_to_pkl({run.query: trimmed_msa[run.query]}, os.path.join(run.pickles, "reference_seq.pkl"))

# Filtering or weighting
with Capturing() as output:
    if run.clustering:  # MSA filtering
        print(f"MSA {run.identity}% identity filtering step")
        msa_num_dict = preprocessor.identity_filtering(msa_num_dict)
        msa, training_alg = preprocessor.get_keys_file_and_np_sequences(msa_num_dict)  # Overlap one treemmer
        seq_weight = np.ones(msa.shape[0])  # we just provide uniform weights for all sequences
    else:  # otherwise the weighting mechanism will be applied
        msa, training_alg = preprocessor.get_keys_file_and_np_sequences(msa_num_dict)  # Overlap one treemmer
        seq_weight = weight_sequences(msa)
msa_log.write("\n".join(output))
train_msa_file_path = os.path.join(run.msa, "training_msa.fasta")
training_alg = MSA.number_to_amino(training_alg)
store_to_fasta(training_alg, train_msa_file_path)
msg = f"Training MSA has {msa.shape[0]} sequences and the width is {msa.shape[1]}\n" \
      f"Training MSA is stored at {train_msa_file_path}"
print(msg)
msa_log.write(msg + f'\n{"=" * 80}\n')

store_to_pkl(seq_weight, os.path.join(run.pickles, "seq_weight.pkl"))
store_to_pkl(training_alg, os.path.join(run.pickles, "training_alignment.pkl"))

# MSA one-hot encoding
binary = MSA.number_to_binary(msa)
store_to_pkl(binary, os.path.join(run.pickles, "seq_msa_binary.pkl"))
msa_log.close()


MSA in ../datasets/EnzymeMiner_PtUGT1_fix.afa loaded
number of sequences: 3437
Trimmed MSA has 3416 sequences and the width is 444
Trimmed MSA is stored at ../results/EnzymeMiner_PtUGT1_DCA_DCP_345/msa/trimmed_msa.fasta
Training MSA has 3416 sequences and the width is 444
Training MSA is stored at ../results/EnzymeMiner_PtUGT1_DCA_DCP_345/msa/training_msa.fasta


In [5]:

with open(run.pickles+ "/msa_columns.pkl", "rb") as input_file:
   msa_columns = pickle.load(input_file)
   
def fix_seqs(input_string):
    seq_ = input_string.replace('.', '')
    seq_ = ''.join([char for char in seq_ if not char.islower()])
    return seq_

def gaplimiter(seq,threshold=0.7):
    gap_percent = seq.count('-') / len(seq)
    if gap_percent > threshold:
        return False
    else:
        return True
    
def gaplimiter_df(df,threshold=0.7):
    return df[df['trimmed_afa'].apply(lambda x: gaplimiter(x,threshold))]

def apply_msa_mask(msa_df, msa_columns):
    msa_df['trimmed_afa'] = msa_df['afa'].apply(lambda x: ''.join([x[i] for i in msa_columns]))
    return msa_df

def hmmer_align(fasta_name,hmm_model,out_path,hmmer_path='/Users/dahala/Projects/HMMER/bin',fasta_ext = 'faa'):
    subprocess.run(f'{hmmer_path}/hmmalign --trim --outformat afa /Users/dahala/GitHub/VAE-enzymes/hmm_model/{hmm_model}.hmm ../datasets/{fasta_name}.{fasta_ext} > {out_path}/hmm/{fasta_name}.afa', shell=True, executable="/bin/zsh")
    return f'{out_path}/hmm/{fasta_name}.afa'

def aligned_df(afa_path,msa_columns=msa_columns):
    dict = {seq.description: fix_seqs(str(seq.seq)) for seq in SeqIO.parse(afa_path, "fasta")}
    df = pd.DataFrame.from_dict(dict, orient='index', columns=['afa'])
    df = apply_msa_mask(df, msa_columns)
    return df

def encode_custom_seqs(fasta_name,hmm_model,out_path,msa_columns=msa_columns,fasta_ext = 'faa',model= f'vae_fold_0.model',batch_size=1):
    afa_path = hmmer_align(fasta_name,hmm_model,out_path,fasta_ext = fasta_ext)
    df = aligned_df(afa_path,msa_columns)
    run.weights = f'{out_path}/model/{model}'
    latent_space = LatentSpace(run)
    mu1_ = []
    mu2_ = []
    for i in range(0, len(df), batch_size):
        latent_embeddings = latent_space.encode(df['trimmed_afa'].to_list()[i:i+batch_size])[0]
        mu1_.append(latent_embeddings[:,0])
        mu2_.append(latent_embeddings[:,1])
    df['mu1'] = np.concatenate(mu1_)
    df['mu2'] = np.concatenate(mu2_)
    return df

def encode_custom_seqs_conditional(fasta_name,hmm_model,out_path,label_list,msa_columns=msa_columns,fasta_ext = 'faa',model= f'vae_fold_0.model'):
    afa_path = hmmer_align(fasta_name,hmm_model,out_path,fasta_ext = fasta_ext)
    df = aligned_df(afa_path,msa_columns)
    run.weights = f'{out_path}/model/{model}'
    latent_space = LatentSpace(run)
    mu1_ = []
    mu2_ = []
    for i in range(0, len(df)):
        encoding = df['trimmed_afa'].to_list()[i]
        label = label_list[i]
        latent_embeddings = latent_space.encode(encoding,c=label)[0]
        mu1_.append(latent_embeddings[:,0])
        mu2_.append(latent_embeddings[:,1])
    df['mu1'] = np.concatenate(mu1_)
    df['mu2'] = np.concatenate(mu2_)
    return df

def encode_custom_seqs_conditional_3D(fasta_name,hmm_model,out_path,label_list,msa_columns=msa_columns,fasta_ext = 'faa',model= f'vae_fold_0.model'):
    afa_path = hmmer_align(fasta_name,hmm_model,out_path,fasta_ext = fasta_ext)
    df = aligned_df(afa_path,msa_columns)
    run.weights = f'{out_path}/model/{model}'
    latent_space = LatentSpace(run)
    mu1_ = []
    mu2_ = []
    mu3_ = []
    for i in range(0, len(df)):
        encoding = df['trimmed_afa'].to_list()[i]
        label = label_list[i]
        latent_embeddings = latent_space.encode(encoding,c=label)[0]
        mu1_.append(latent_embeddings[:,0])
        mu2_.append(latent_embeddings[:,1])
        mu3_.append(latent_embeddings[:,2])
    df['mu1'] = np.concatenate(mu1_)
    df['mu2'] = np.concatenate(mu2_)
    df['mu3'] = np.concatenate(mu3_)
    return df

#### Conditional labels
Before starting the training process, ensure that your labels are properly preprocessed if you intend to use them for **conditional training**. 

This involves converting your labels into a suitable format, such as one-hot or integer encoding (see and modify **custom_classifier** function). 

Additionally, specify the number of classes represented by your labels.

In [53]:
# Specify number of classes
N_CLASSES = 2
def parse_conditional_labels_to_categories(file_name):
    """ Parse conditional label file """
    # Please specify your classification function to the bins, the interface of function must stay the same, example
    def custom_classifier(prob_vals) -> torch.Tensor:
        label = torch.zeros(N_CLASSES)
        for i,val in enumerate(prob_vals):
            if val > run.prob_threshold:
                label[i] = 1
        return label
    
    label_dict = {}
    prob_df = pd.read_csv(file_name,delimiter='\t',header=0)
    for acc,label_vals in zip(prob_df.enzyme,prob_df.iloc[:,1:].values):
            if label_vals is None:
                print(f"Error while parsing labels, accession {acc} does not have labels")
                exit(1)
            label_dict[acc] = custom_classifier(label_vals)
    return label_dict

""" In the case of conditional model please preprocess labels """
label_file_path = run.conditional_labels  # you can specify it in the configuration path or provide custom one 

# Now get labels from your file, simple txt or similar file with sequence accession\tlabel_value pair per line (modify function as needed)  
label_dict = parse_conditional_labels_to_categories(label_file_path)

labels_file_pkl = os.path.join(run.conditional_data, "conditional_labels_to_categories.pkl")
msa_keys = load_from_pkl(os.path.join(run.pickles, "keys_list.pkl"))

# get labels to the similar order as in msa (just in case, to enable using indexes in training method)
msa_key_label_order = {}
array_of_labels = torch.zeros(len(msa_keys), N_CLASSES)
for label_i, msa_key in enumerate(msa_keys):
    try:
        msa_key_fix = msa_key.split()[0]
        msa_key_label_order[msa_key] = label_dict[msa_key_fix]
        array_of_labels[label_i] = label_dict[msa_key_fix]
    except KeyError:
        print(f"{msa_key} is not in the label_dict, please provide labels for all sequences")
        exit(1)
        
# now store in both raw and text version
store_to_pkl(array_of_labels, labels_file_pkl)
with open(os.path.join(run.conditional_data, "msa_key_label_order.txt"), 'w') as txt_labels:
    for k, label_tensor in msa_key_label_order.items():
        txt_labels.write(f"{k} {label_tensor}\n")

### Training of the model
Select the models you would like to work with and prepare data for training and benchmarking if selected in configuration file. 

If you select a conditional model please see above cell and prepare labels for it.  

In [ ]:
""" Configure model """
available_models = ["vae", "vae_conditional"]
model_selected = available_models[1]

""" Prepare data for training """
setup_train(run, model_selected)
model = Train(run)

In [ ]:
"""
Train VAE with a preprocessed MSA
:param conf_path: path to the configuration file of the project
:return:
"""
# logging
train_log = open(os.path.join(run.logs, 'train_log.txt'), "w")
model = Train(run)
with Capturing() as output_train:
    model.train()
train_log.write(" Training \n" + "\n".join(output_train) + f'\n{"=" * 80}\n')
train_log.close()

In [ ]:
""" Do I work with conditional model? """

load_from_pkl(os.path.join(run.model, "model_params.pkl"))['model']

### Generative Capacity evaluation
If you wish to obtain estimates of your model's generative capacity, please set run_capacity_test to true in the configuration file before starting the training.

This test generates first and second order statistics of your model.

In [ ]:
%cd /Users/dahala/GitHub/VAE-enzymes/vae_notebooks
from notebooks.minimal_version.benchmark import Benchmarker
%cd /Users/dahala/GitHub/VAE-enzymes/vae_notebooks/notebooks/
run.weights = f'{run.results[:-7]}model/vae_fold_0_e3999.model'
""" 
Benchmark model 
How well our model can regenarate sequences in various sets (positive, negative and training)
"""
N_SAMPLES = 500

Benchmarker(run, samples=N_SAMPLES).bench_dataset()


### Projection to the latent space
Project all points to the latent space and make custom selection of sequences

In [ ]:
%cd /Users/dahala/GitHub/VAE-enzymes/vae_notebooks
from matplotlib import pyplot as plt
from notebooks.minimal_version.latent_space import LatentSpace
%cd /Users/dahala/GitHub/VAE-enzymes/vae_notebooks/notebooks/
for epoch in range(1,15000):
    if (epoch + 1) % 500 == 0:
        # Projection to the latent space
        # Show your queries
        run.weights = f'{run.results[:-7]}model/vae_fold_0_e{epoch}.model'
        latent_space = LatentSpace(run)
        msa_embeddings = latent_space.msa_embeddings["mus"]


        query_coords = latent_space.key_to_embedding(run.fixed_sequences)

        custom_sequences = []  # give keys to MSA and embed them to the latent space


        fig_lat, ax = plt.subplots(1, 1)

        ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, )
        ax.plot(query_coords[0::2], query_coords[1::2], '.', color='red')

        # Project 
        for seq_id, seq_mu in enumerate(custom_sequences):
            ax.plot(seq_mu[0], seq_mu[1], '.', color='black', alpha=1, markersize=5,
                    label='({})'.format(seq_id))
            
            ax.annotate(str(seq_id), (seq_mu[0], seq_mu[1]))
        ax.set_xlabel("$Z_1$")
        ax.set_ylabel("$Z_2$")
        ax.set_title(f"Latent space projection epoch {epoch}")
        fig_lat.savefig(os.path.join(run.results, f"latent_space_e{epoch}.png"))

In [ ]:
%cd /Users/dahala/GitHub/VAE-enzymes/vae_notebooks
from matplotlib import pyplot as plt
from notebooks.minimal_version.latent_space import LatentSpace
%cd /Users/dahala/GitHub/VAE-enzymes/vae_notebooks/notebooks/

# Projection to the latent space
# Show your queries
run.weights = f'{run.result}/model/vae_fold_0.model'
latent_space = LatentSpace(run)
msa_embeddings = latent_space.msa_embeddings["mus"]


query_coords = latent_space.key_to_embedding(run.fixed_sequences)

custom_sequences = []  # give keys to MSA and embed them to the latent space

conditional = True
fig_lat, ax = plt.subplots(1, 1)
if conditional:
    for i,label in enumerate(msa_key_label_order.values()):
        if label[0] == 0:
        #if label[0] == 1:
            ax.plot(msa_embeddings[i, 0], msa_embeddings[i, 1], '.', alpha=0.1, markersize=3, color='red')
        else:
            ax.plot(msa_embeddings[i, 0], msa_embeddings[i, 1], '.', alpha=0.1, markersize=3, color='blue')
    ax.plot(query_coords[0::2], query_coords[1::2], '.', color='k')
else:
    ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, )
    ax.plot(query_coords[0::2], query_coords[1::2], '.', color='red')

# Project 
for seq_id, seq_mu in enumerate(custom_sequences):
    ax.plot(seq_mu[0], seq_mu[1], '.', color='black', alpha=1, markersize=5,
            label='({})'.format(seq_id))
    
    ax.annotate(str(seq_id), (seq_mu[0], seq_mu[1]))
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")
ax.set_title(f"Latent space projection final")
if conditional:
    fig_lat.savefig(os.path.join(run.results, f"latent_space_conditional.png"))
else:
    fig_lat.savefig(os.path.join(run.results, f"latent_space.png"))

### 3D Projection to the latent space
Project all points to the latent space and make custom selection of sequences

In [ ]:
%cd /Users/dahala/GitHub/VAE-enzymes/vae_notebooks
from matplotlib import pyplot as plt
from notebooks.minimal_version.latent_space import LatentSpace
%cd /Users/dahala/GitHub/VAE-enzymes/vae_notebooks/notebooks/

# Projection to the latent space
# Show your queries
#run.weights = f'{run.result}/model/vae_fold_0_e9999.model'
latent_space = LatentSpace(run)
msa_embeddings = latent_space.msa_embeddings["mus"]


query_coords = latent_space.key_to_embedding(run.fixed_sequences)


conditional = True
# Prepare arrays to store points for each label
red_points = []
blue_points = []

if conditional:
    for i, label in enumerate(msa_key_label_order.values()):
        if label[0] == 0:
        #if label[0] == 1:
            red_points.append(msa_embeddings[i].numpy())
        else:
            blue_points.append(msa_embeddings[i].numpy())

# Convert to NumPy arrays for easier handling
red_points = np.array(red_points)
blue_points = np.array(blue_points)

angles = [(20, 30), (20, 80), (20, 130), (20, 180), (20, 240), (90, 0)]

# Create a figure with 4 subplots for different angles
fig = plt.figure(figsize=(24, 18))

for idx, (elev, azim) in enumerate(angles):
    ax = fig.add_subplot(2, 3, idx + 1, projection='3d')

    # Plot all red points in one scatter call
    if red_points.size > 0:
        ax.scatter(red_points[:, 0], red_points[:, 1], red_points[:, 2], alpha=1, s=3, color='red')

    # Plot all blue points in one scatter call
    if blue_points.size > 0:
        ax.scatter(blue_points[:, 0], blue_points[:, 1], blue_points[:, 2], alpha=0.1, s=3, color='blue')

    # Plot query coordinates
    ax.scatter(query_coords[0::3], query_coords[1::3], query_coords[2::3], color='k', s=20)

    # Set labels and title for each subplot
    ax.set_xlabel("$Z_1$")
    ax.set_ylabel("$Z_2$")
    ax.set_zlabel("$Z_3$")
    ax.set_title(f"View (elev={elev}, azim={azim})")
    
    # Set the view angle
    ax.view_init(elev=elev, azim=azim)

# Adjust layout
plt.tight_layout()
plt.savefig(os.path.join(run.results, f"latent_space_3d.png"))
plt.show()

### Generate ancestors
Using straight evolution protocol  

In [ ]:
from notebooks.minimal_version.evolution_protocols.straight_evolution import StraightEvolution
#run.weights = f'../results/EnzymeMiner_PtUGT1/model/vae_fold_0_e3999.model'
latent_space = LatentSpace(run)
msa_embeddings = latent_space.msa_embeddings["mus"]
protocol = StraightEvolution(run)
ancestors = protocol.get_ancestors(profile=False)  # FIXME

In [10]:
import sys, importlib
importlib.reload(sys.modules['notebooks.minimal_version.latent_space'])
from notebooks.minimal_version.latent_space import LatentSpace

In [11]:
fireprot_ancestors = {}
for i in range(151,300):
    seq = np.loadtxt(f'../../FireProtASR_results/ancestrals/node_{i}.fas',dtype=str)
    seq = str(seq)
    fireprot_ancestors[f'anc{i}'] = seq

In [ ]:
hmm_name = 'PFAM201'#'GASP'#'EnzymeMiner_PtUGT1'
fireprot_ancestors_df = encode_custom_seqs('At71B1_ancestral_sequences',hmm_name,run.root_dir,fasta_ext='fasta')#,model= f'vae_fold_0_e6499.model',batch_size=1)

In [ ]:
fireprot_ancestors_df

In [14]:
query_ancestors_df = fireprot_ancestors_df.loc[['anc_294','anc_293','anc_292','anc_289','anc_288','anc_209','anc_200','anc_196','anc_151']]

In [ ]:
run.fixed_sequences

In [ ]:
fig_lat, ax = plt.subplots(1, 1)

ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, color='lightgray')


# Project 
i = 0
for seq_id, seq in ancestors.items():
    seq = ''.join(seq)
    seq_mu = latent_space.encode(seq)[0][0]
    if i == 0:
        ax.plot(seq_mu[0], seq_mu[1], '.', color='red', alpha=0.2, markersize=5,
            label='VAE ancestors')
        i += 1
    else:
        ax.plot(seq_mu[0], seq_mu[1], '.', color='red', alpha=0.2, markersize=5)

i = 0
for mu1,mu2 in query_ancestors_df[['mu1','mu2']].values:
    if i == 0:
        ax.plot(mu1, mu2, '.', color='blue', alpha=0.2, markersize=5,
            label='FireProt ancestors')
        i += 1
    else:
        ax.plot(mu1, mu2, '.', color='blue', alpha=0.2, markersize=5)

ax.scatter(query_coords[0], query_coords[1], color='white',edgecolor='k', s=50, label='Query', zorder=10)
ax.legend()
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")
fig_lat.savefig(os.path.join(run.results, "latent_space_with_anc.png"))

### Sequence Projection
Projecting custom sequence to latent space

In [ ]:
run.result

In [ ]:
latent_space = LatentSpace(run)
#hmm_name = 'PFAM201'
hmm_name = 'GASP'
#hmm_name = 'EnzymeMiner_PtUGT1'
known_df_ = encode_custom_seqs('KnownGT1',hmm_name,run.result,fasta_ext='faa',model= f'vae_fold_0.model',batch_size=1)

In [74]:
KnownUGT_df = pd.read_excel('../../UGT_names_excel.xlsx',index_col=2)
KnownUGT_df = KnownUGT_df.merge(known_df_, left_index=True, right_index=True)

In [139]:
All_data_df = pd.read_csv('../../All_data.csv')

In [ ]:
All_data_unique = All_data_df.drop_duplicates(subset='Accession', keep='first')
with open('../datasets/All_data_unique.fasta', 'w') as f:
    for index, row in All_data_unique.iterrows():
        if not row['Official_name'].count('+') == 1:
            f.write(f">{row['Official_name']}\n{row['Sequence']}\n")
all_encode = encode_custom_seqs('All_data_unique',hmm_name,run.result,fasta_ext='fasta',model= f'vae_fold_0.model',batch_size=1)
All_data_unique = All_data_unique.merge(all_encode, left_on='Official_name', right_index=True)

In [ ]:
Dataset1 = All_data_df[All_data_df.ID=='Dataset1']
with open('../datasets/Dataset1.faa','w') as f:
    for enz,seq in zip(Dataset1['Official_name'],Dataset1['Sequence']):
        f.write(f'>{enz}\n{seq}\n')
dataset1_encode = encode_custom_seqs('Dataset1',hmm_name,run.result,fasta_ext='faa',model= f'vae_fold_0.model',batch_size=1)
Dataset1 = Dataset1.merge(dataset1_encode, left_on='Official_name', right_index=True)
Dataset1_test = Dataset1[Dataset1.Acceptor.isin(['3,4-Dichloroaniline', '3,4-Dichlorothiophenol', 'Resveratrol',
                                                  'Cinnamyl alcohol', 'Quercetin', 'Cinnamic acid', 'Linalool',
                                                    'Ferulic acid', 'Capsaicin', 'Naringenin'])]

In [ ]:
DON_GT1_kin = All_data_df[All_data_df.ID=='DON_GT1_kin']
DON_GT1_kin_unique = DON_GT1_kin.drop_duplicates(subset='Official_name')
with open('../datasets/DON_GT1_kin.faa','w') as f:
    for enz,seq in zip(DON_GT1_kin_unique.Official_name,DON_GT1_kin_unique.Sequence):
        f.write(f'>{enz}\n{seq}\n')
don_gt1_kin_df = encode_custom_seqs('DON_GT1_kin',hmm_name,run.result,fasta_ext='faa',model= f'vae_fold_0.model',batch_size=1)
DON_GT1_kin_unique = DON_GT1_kin_unique.merge(don_gt1_kin_df, left_on='Official_name', right_index=True)

In [ ]:
PtUGT1_TMmut = All_data_df[(All_data_df.ID == 'PtUGT1_TMmut')&(All_data_df.Measurement_type=='Tm')]

with open('../datasets/PtUGT1_TMmut.faa', 'w') as f:
    for enz,mut, seq in zip(PtUGT1_TMmut['Official_name'],PtUGT1_TMmut['Mutation_position'], PtUGT1_TMmut['Sequence']):
        f.write(f'>{enz} | {mut}\n{seq}\n')

PtUGT1_TMmut_encode = encode_custom_seqs('PtUGT1_TMmut',hmm_name,run.result,fasta_ext='faa',model= f'vae_fold_0.model',batch_size=1)
for col in PtUGT1_TMmut_encode.columns:
    PtUGT1_TMmut[col] = PtUGT1_TMmut_encode[col].values

In [ ]:
CGT_sol = All_data_df[All_data_df.ID=='CGT_sol']
with open('../datasets/CGT_sol.faa','w') as f:
    for enz,seq in zip(CGT_sol['Official_name'],CGT_sol['Sequence']):
        f.write(f'>{enz}\n{seq}\n')
cgt_sol_df = encode_custom_seqs('CGT_sol',hmm_name,run.result,fasta_ext='faa',model= f'vae_fold_0.model',batch_size=1)
CGT_sol = CGT_sol.merge(cgt_sol_df, left_on='Official_name', right_index=True)

In [ ]:
Anthranilates_GT1 = All_data_df[(All_data_df.ID == 'Anthranilates_GT1')]
with open('../datasets/Anthranilates_GT1.faa', 'w') as f:
    for enz, seq in zip(Anthranilates_GT1['Official_name'], Anthranilates_GT1['Sequence']):
        if enz != 'Sl_UGT72B68+GmSuSy':
            f.write(f'>{enz}\n{seq}\n')
anthranilates_gt1_encode = encode_custom_seqs('Anthranilates_GT1',hmm_name,run.result,fasta_ext='faa',model= f'vae_fold_0.model',batch_size=1)
Anthranilates_GT1 = Anthranilates_GT1.merge(anthranilates_gt1_encode, left_on='Official_name', right_index=True)

In [82]:
KnownGT_colors = {}
okabe_ito_colors_ext = ['white','lightgray','#000000','#750000','#56B4E9','#E69F00','#009E73','#D55E00','#0072B2','#CC79A7','#F0E442','darkgray']
colors=okabe_ito_colors_ext[3:]+['darkgreen']+['#FF6865']
for i,phylum in enumerate(pd.unique(KnownUGT_df.Phylum)):
    KnownGT_colors[phylum] = colors[i]

fig_lat, ax = plt.subplots(1, 1)

ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, zorder=1,color='darkgray')


Project
for mu1,mu2 in query_df[['mu1','mu2']].values:
    ax.scatter(mu1, mu2, color='lightblue', alpha=1, s=100, zorder=2,edgecolor='black')

#ax.scatter(seq_mu[0], seq_mu[1], color='green', alpha=1, s=100,edgecolor='black', zorder=2)
    
    #ax.annotate(str(seq_id), (seq_mu[0], seq_mu[1]))
ax.scatter(query_coords[0::2], query_coords[1::2], color='gold', alpha=1, s=20,edgecolor='black', zorder=5)
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")
ax.legend(fontsize=10)
fig_lat.savefig(os.path.join(run.results, "latent_space_with_custom_seq.png"))

In [ ]:
fig_lat, ax = plt.subplots(figsize=(6,6))
# Project
msa_embeddings = latent_space.msa_embeddings["mus"]
ax.scatter(All_data_unique.mu1, All_data_unique.mu2, color='gray', alpha=1, s=30,edgecolor='black', zorder=3)
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")

ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, zorder=1,color='blue')
#ax.set_xlim(-10,5)
#ax.set_ylim(-5,11)
ax.set_box_aspect(1)
plt.tight_layout()
fig_lat.savefig(os.path.join(run.results, "latent_space_with_exp_enz.png"))

In [ ]:
for sub in ['3,4-Dichloroaniline', '3,4-Dichlorothiophenol', 'Resveratrol',
                                                  'Cinnamyl alcohol', 'Quercetin', 'Cinnamic acid', 'Linalool',
                                                    'Ferulic acid', 'Capsaicin', 'Naringenin']:
    print(All_data_df[All_data_df.Acceptor==sub].Acceptor.unique(),All_data_df[All_data_df.Acceptor==sub].Acceptor_cid.unique())

In [ ]:
fig_lat, axes = plt.subplots(2, 5,figsize=(20,10))
Dataset1_test_bin = Dataset1_test[Dataset1_test.Measurement_type=='Activity_score']
# Project
for ax,substrate in zip(axes.flatten(),['3,4-Dichloroaniline', '3,4-Dichlorothiophenol', 'Resveratrol',
                                                  'Cinnamyl alcohol', 'Quercetin', 'Cinnamic acid', 'Linalool',
                                                    'Ferulic acid', 'Capsaicin', 'Naringenin']):
    subset_ = Dataset1_test_bin[(Dataset1_test_bin.Acceptor==substrate)]
    ax.scatter(subset_.mu1, subset_.mu2, c=subset_.Measurement_val.to_numpy(dtype=float), alpha=1, s=20,edgecolor='black', zorder=3,cmap='coolwarm')
    ax.set_title(substrate)
    ax.set_xlabel("$Z_1$")
    ax.set_ylabel("$Z_2$")
    ax.set_aspect('equal')
    ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, zorder=1,color='darkgray')
plt.tight_layout()
fig_lat.savefig(os.path.join(run.results, "latent_space_with_dataset1.png"))


In [ ]:
fig_lat, ax = plt.subplots(1, 2,figsize=(10,5))

# Project
ax[0].scatter(DON_GT1_kin_unique.mu1, DON_GT1_kin_unique.mu2, c=DON_GT1_kin[DON_GT1_kin.Measurement_type=='Km'].Measurement_val.to_numpy(dtype=float), alpha=1, s=20,edgecolor='black', zorder=2,label='DON_GT1_Km')
ax[1].scatter(DON_GT1_kin_unique.mu1, DON_GT1_kin_unique.mu2, c=DON_GT1_kin[DON_GT1_kin.Measurement_type=='kcat'].Measurement_val.to_numpy(dtype=float), alpha=1, s=20,edgecolor='black', zorder=2,label='DON_GT1_kcat')
for i in range(2):
    ax[i].plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, zorder=1,color='darkgray')
    ax[i].set_xlabel("$Z_1$")
    ax[i].set_ylabel("$Z_2$")
    ax[i].legend(fontsize=10)
    ax[i].set_box_aspect(1)
    #ax[i].set_ylim(0,5)
    #ax[i].set_xlim(-10,0)
fig_lat.savefig(os.path.join(run.results, "latent_space_with_DON_GT1_kin.png"))

In [ ]:
fig_lat, ax = plt.subplots(figsize=(6,6))
# Project

ax.scatter(PtUGT1_TMmut.mu1, PtUGT1_TMmut.mu2, c=PtUGT1_TMmut.Measurement_val.to_numpy(dtype=float), alpha=1, s=30,edgecolor='black', zorder=3)
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")

#ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, zorder=1,color='darkgray')
#ax.set_xlim(-4.3,-4.35)
#ax.set_ylim(PtUGT1_TMmut.mu2.min()*0.99,0.34)
ax.set_box_aspect(1)
plt.tight_layout()
fig_lat.savefig(os.path.join(run.results, "latent_space_with_PtUGT1_TMmut.png"))

In [ ]:
fig_lat, ax = plt.subplots(figsize=(6,6))
# Project

ax.scatter(PtUGT1_TMmut.mu1, PtUGT1_TMmut.mu2, c=PtUGT1_TMmut.Mutation_position.str.count(','), alpha=1, s=30,edgecolor='black', zorder=3)
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")

#ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, zorder=1,color='darkgray')
#ax.set_xlim(-4.3,-4.35)
#ax.set_ylim(PtUGT1_TMmut.mu2.min()*0.99,0.34)
ax.set_box_aspect(1)
plt.tight_layout()
fig_lat.savefig(os.path.join(run.results, "latent_space_with_PtUGT1_TMmut_mutnumb.png"))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr

# Prepare the data
#X = PtUGT1_TMmut[['mu1', 'mu2']]
#y = PtUGT1_TMmut['Measurement_val'].astype(float)  # Ensure the target variable is float

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = PtUGT1_TMmut[PtUGT1_TMmut.Mutation_position.str.count(',')<1][['mu1', 'mu2']]
y_train = PtUGT1_TMmut[PtUGT1_TMmut.Mutation_position.str.count(',')<1]['Measurement_val'].astype(float)
X_test = PtUGT1_TMmut[PtUGT1_TMmut.Mutation_position.str.count(',')>0][['mu1', 'mu2']]
y_test = PtUGT1_TMmut[PtUGT1_TMmut.Mutation_position.str.count(',')>0]['Measurement_val'].astype(float)
# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
pearson = pearsonr(y_test, y_pred)
spearman = spearmanr(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")
print(f"Pearson Correlation: {pearson[0]}")
print(f"Spearman Correlation: {spearman[0]}")

# Display the coefficients
print(f"Coefficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")

In [ ]:
fig,axis = plt.subplots(1,1,figsize=(6,6))
# Plot the data
axis.scatter(y_test, y_pred, c=PtUGT1_TMmut.loc[y_test.index].Mutation_position.str.count(','))
axis.set_xlabel("True Values")
axis.set_ylabel("Predictions")
axis.set_box_aspect(1)
axis.plot([0, 1], [1, 0], transform=axis.transAxes, ls="--", c="grey")
plt.savefig(os.path.join(run.results, "regression_PtUGT1_TMmut.png"))


In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(PtUGT1_TMmut.Mutation_position.str.count(','),PtUGT1_TMmut.Measurement_val.to_numpy(dtype=float))
plt.xlabel('Number of mutations')
plt.ylabel('Measurement value')
pearsonr(PtUGT1_TMmut.Mutation_position.str.count(','),PtUGT1_TMmut.Measurement_val.to_numpy(dtype=float))[0]

In [ ]:
fig_lat, ax = plt.subplots(figsize=(6,6))
# Project
CGT_sol = CGT_sol.sort_values(by='Measurement_val')
ax.scatter(CGT_sol.mu1, CGT_sol.mu2, c=CGT_sol.Measurement_val.to_numpy(dtype=float), alpha=1, s=30,edgecolor='black', zorder=3)
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")

ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, zorder=1,color='darkgray')
#ax.set_xlim(-10,5)
#ax.set_ylim(-5,11)
ax.set_box_aspect(1)
plt.tight_layout()
fig_lat.savefig(os.path.join(run.results, "latent_space_with_CGT_sol.png"))

In [ ]:
fig_lat, axes = plt.subplots(2, 3,figsize=(15,10))
# Project
for ax,substrate in zip(axes.flatten(),Anthranilates_GT1.Acceptor.unique()):
    subset_ = Anthranilates_GT1[(Anthranilates_GT1.Acceptor==substrate)]
    ax.scatter(subset_.mu1, subset_.mu2, c=subset_.Measurement_val.to_numpy(dtype=float), alpha=1, s=20,edgecolor='black', zorder=3)
    ax.set_title(substrate)
    ax.set_xlabel("$Z_1$")
    ax.set_ylabel("$Z_2$")
    ax.set_aspect('equal')
    ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, zorder=1,color='darkgray')
plt.tight_layout()
fig_lat.savefig(os.path.join(run.results, "latent_space_with_Anthranilates_GT1.png"))

In [ ]:
fig_lat, ax = plt.subplots(1, 1)

ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, zorder=1,color='darkgray')


# Project
for phylum in pd.unique(KnownUGT_df.Phylum):
    df_ = gaplimiter_df(KnownUGT_df[KnownUGT_df.Phylum == phylum],threshold=0.8)
    if df_.shape[0] == 0:
        continue
    mu1_ = df_.mu1
    mu2_ = df_.mu2
    ax.scatter(mu1_, mu2_, color=KnownGT_colors[phylum], alpha=1, s=50,edgecolor='black', zorder=2, label = phylum)
    #ax.annotate(str(seq_id), (seq_mu[0], seq_mu[1]))
#ax.scatter(query_coords[0::2], query_coords[1::2], color='red', alpha=1, s=20,edgecolor='black', zorder=2)
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")
ax.legend(fontsize=10)
fig_lat.savefig(os.path.join(run.results, "latent_space_with_knownGT1_seq.png"))

### Sequence Projection for C-VAE

In [ ]:
All_data_df = pd.read_csv('../../All_data.csv')
Dataset1 = All_data_df[All_data_df.ID=='Dataset1']
latent_space = LatentSpace(run)
hmm_name = 'EnzymeMiner_PtUGT1'#'PFAM201''GASP'

#### 2D

In [ ]:
Dataset1_Dichloroaniline_bin = Dataset1[(Dataset1.Acceptor=='3,4-Dichloroaniline')&(Dataset1.Measurement_type=='Activity_score')]
Dataset1_Dichloroaniline_labels = torch.tensor([[1,0] if float(val) == 1 else [0,1] for val in Dataset1_Dichloroaniline_bin['Measurement_val']])
Dataset1_Dichloroaniline_labels_neg = torch.tensor([[1,0] if float(val) == 0 else [0,1] for val in Dataset1_Dichloroaniline_bin['Measurement_val']])
with open('../datasets/Dataset1_Dichloroaniline.faa','w') as f:
    for enz,seq in zip(Dataset1_Dichloroaniline_bin['Official_name'],Dataset1_Dichloroaniline_bin['Sequence']):
        f.write(f'>{enz}\n{seq}\n')
dataset1_bin_encode = encode_custom_seqs_conditional('Dataset1_Dichloroaniline',hmm_name,run.result,Dataset1_Dichloroaniline_labels,fasta_ext='faa',model= f'vae_fold_0.model')
Dataset1_Dichloroaniline_bin = Dataset1_Dichloroaniline_bin.merge(dataset1_bin_encode, left_on='Official_name', right_index=True)
dataset1_bin_encode_neg = encode_custom_seqs_conditional('Dataset1_Dichloroaniline',hmm_name,run.result,Dataset1_Dichloroaniline_labels_neg,fasta_ext='faa',model= f'vae_fold_0.model')
Dataset1_Dichloroaniline_bin = Dataset1_Dichloroaniline_bin.merge(dataset1_bin_encode_neg[['mu1','mu2']], left_on='Official_name', right_index=True,suffixes=('','_neg'))

In [ ]:
Dataset1_DCP_bin = Dataset1[(Dataset1.Acceptor=='3,4-Dichlorothiophenol')&(Dataset1.Measurement_type=='Activity_score')]
Dataset1_DCP_labels = torch.tensor([[1,0] if float(val) == 1 else [0,1] for val in Dataset1_DCP_bin['Measurement_val']])
Dataset1_DCP_labels_neg = torch.tensor([[1,0] if float(val) == 0 else [0,1] for val in Dataset1_DCP_bin['Measurement_val']])
with open('../datasets/Dataset1_DCP.faa','w') as f:
    for enz,seq in zip(Dataset1_DCP_bin['Official_name'],Dataset1_DCP_bin['Sequence']):
        f.write(f'>{enz}\n{seq}\n')
dataset1_bin_encode = encode_custom_seqs_conditional('Dataset1_DCP',hmm_name,run.result,Dataset1_DCP_labels,fasta_ext='faa',model= f'vae_fold_0.model')
Dataset1_DCP_bin = Dataset1_DCP_bin.merge(dataset1_bin_encode, left_on='Official_name', right_index=True)
dataset1_bin_encode_neg = encode_custom_seqs_conditional('Dataset1_DCP',hmm_name,run.result,Dataset1_DCP_labels_neg,fasta_ext='faa',model= f'vae_fold_0.model')
Dataset1_DCP_bin = Dataset1_DCP_bin.merge(dataset1_bin_encode_neg[['mu1','mu2']], left_on='Official_name', right_index=True,suffixes=('','_neg'))

In [ ]:
fig_lat, ax = plt.subplots(figsize=(6,6))
# Project
ax.scatter(Dataset1_Dichloroaniline_bin.mu1, Dataset1_Dichloroaniline_bin.mu2, c=Dataset1_Dichloroaniline_bin.Measurement_val.to_numpy(dtype=float), alpha=1, s=30,edgecolor='black', zorder=3,cmap='coolwarm')
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")

conditional = True
if conditional:
    for i,label in enumerate(msa_key_label_order.values()):
        if label[0] == 1:
        #if label[0] == 0:
            ax.plot(msa_embeddings[i, 0], msa_embeddings[i, 1], '.', alpha=0.1, markersize=3, color='red')
        else:
            ax.plot(msa_embeddings[i, 0], msa_embeddings[i, 1], '.', alpha=0.1, markersize=3, color='blue')
    ax.plot(query_coords[0::2], query_coords[1::2], '.', color='k')
else:
    ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, )
    ax.plot(query_coords[0::2], query_coords[1::2], '.', color='red')
#ax.set_xlim(-3,3)
#ax.set_ylim(-5,2)
ax.set_box_aspect(1)
plt.tight_layout()
fig_lat.savefig(os.path.join(run.results, "latent_space_with_D1_Dichloroaniline_bin.png"))

In [ ]:
fig_lat, ax = plt.subplots(figsize=(6,6))
# Project
ax.scatter(Dataset1_DCP_bin.mu1, Dataset1_DCP_bin.mu2, c=Dataset1_DCP_bin.Measurement_val.to_numpy(dtype=float), alpha=1, s=30,edgecolor='black', zorder=3,cmap='coolwarm')
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")

conditional = True
if conditional:
    for i,label in enumerate(msa_key_label_order.values()):
        if label[0] == 1:
        #if label[0] == 0:
            ax.plot(msa_embeddings[i, 0], msa_embeddings[i, 1], '.', alpha=0.1, markersize=3, color='red')
        else:
            ax.plot(msa_embeddings[i, 0], msa_embeddings[i, 1], '.', alpha=0.1, markersize=3, color='blue')
    ax.plot(query_coords[0::2], query_coords[1::2], '.', color='k')
else:
    ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, )
    ax.plot(query_coords[0::2], query_coords[1::2], '.', color='red')
#ax.set_xlim(-3,3)
#ax.set_ylim(-5,11)
ax.set_box_aspect(1)
plt.tight_layout()
fig_lat.savefig(os.path.join(run.results, "latent_space_with_D1_DCP_bin.png"))

In [ ]:
fig_lat, ax = plt.subplots(figsize=(6,6))
# Project
Dataset1_Dichloroaniline_bin_subset = Dataset1_Dichloroaniline_bin[Dataset1_Dichloroaniline_bin.Measurement_val.to_numpy(dtype=float)==1]
ax.scatter(Dataset1_Dichloroaniline_bin_subset.mu1, Dataset1_Dichloroaniline_bin_subset.mu2, alpha=0.5, s=30,edgecolor='black', zorder=3,cmap='red')
ax.scatter(Dataset1_Dichloroaniline_bin_subset.mu1_neg, Dataset1_Dichloroaniline_bin_subset.mu2_neg, alpha=0.5, s=30,edgecolor='black', zorder=3,cmap='blue')
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")

conditional = True
if conditional:
    for i,label in enumerate(msa_key_label_order.values()):
        #if label[0] == 0:
        if label[0] == 1:
            ax.plot(msa_embeddings[i, 0], msa_embeddings[i, 1], '.', alpha=0.1, markersize=3, color='red')
        else:
            ax.plot(msa_embeddings[i, 0], msa_embeddings[i, 1], '.', alpha=0.1, markersize=3, color='blue')
    ax.plot(query_coords[0::2], query_coords[1::2], '.', color='k')
else:
    ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, )
    ax.plot(query_coords[0::2], query_coords[1::2], '.', color='red')
#ax.set_xlim(-3,3)
#ax.set_ylim(-5,2)
ax.set_box_aspect(1)
plt.tight_layout()
fig_lat.savefig(os.path.join(run.results, "latent_space_with_D1_Dichloroaniline_bin_pos_reversed.png"))

In [ ]:
fig_lat, ax = plt.subplots(figsize=(6,6))
# Project
Dataset1_DCP_bin_subset = Dataset1_DCP_bin[Dataset1_DCP_bin.Measurement_val.to_numpy(dtype=float)==1]
ax.scatter(Dataset1_DCP_bin_subset.mu1, Dataset1_DCP_bin_subset.mu2, alpha=0.5, s=30,edgecolor='black', zorder=3,cmap='red')
ax.scatter(Dataset1_DCP_bin_subset.mu1_neg, Dataset1_DCP_bin_subset.mu2_neg, alpha=0.5, s=30,edgecolor='black', zorder=3,cmap='navy')
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")

conditional = True
if conditional:
    for i,label in enumerate(msa_key_label_order.values()):
        if label[0] == 1:
        #if label[0] == 0:
            ax.plot(msa_embeddings[i, 0], msa_embeddings[i, 1], '.', alpha=0.1, markersize=3, color='red')
        else:
            ax.plot(msa_embeddings[i, 0], msa_embeddings[i, 1], '.', alpha=0.1, markersize=3, color='blue')
    ax.plot(query_coords[0::2], query_coords[1::2], '.', color='k')
else:
    ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, )
    ax.plot(query_coords[0::2], query_coords[1::2], '.', color='red')
#ax.set_xlim(-3,3)
#ax.set_ylim(-5,2)
ax.set_box_aspect(1)
plt.tight_layout()
fig_lat.savefig(os.path.join(run.results, "latent_space_with_D1_DCP_bin_pos_reversed.png"))

#### 3D

In [ ]:
Dataset1_Dichloroaniline_bin = Dataset1[(Dataset1.Acceptor=='3,4-Dichloroaniline')&(Dataset1.Measurement_type=='Activity_score')]
Dataset1_Dichloroaniline_labels = torch.tensor([[1,0] if float(val) == 1 else [0,1] for val in Dataset1_Dichloroaniline_bin['Measurement_val']])
Dataset1_Dichloroaniline_labels_neg = torch.tensor([[1,0] if float(val) == 0 else [0,1] for val in Dataset1_Dichloroaniline_bin['Measurement_val']])
with open('../datasets/Dataset1_Dichloroaniline.faa','w') as f:
    for enz,seq in zip(Dataset1_Dichloroaniline_bin['Official_name'],Dataset1_Dichloroaniline_bin['Sequence']):
        f.write(f'>{enz}\n{seq}\n')
dataset1_bin_encode = encode_custom_seqs_conditional_3D('Dataset1_Dichloroaniline',hmm_name,run.result,Dataset1_Dichloroaniline_labels,fasta_ext='faa',model= f'vae_fold_0.model')
Dataset1_Dichloroaniline_bin = Dataset1_Dichloroaniline_bin.merge(dataset1_bin_encode, left_on='Official_name', right_index=True)
dataset1_bin_encode_neg = encode_custom_seqs_conditional_3D('Dataset1_Dichloroaniline',hmm_name,run.result,Dataset1_Dichloroaniline_labels_neg,fasta_ext='faa',model= f'vae_fold_0.model')
Dataset1_Dichloroaniline_bin = Dataset1_Dichloroaniline_bin.merge(dataset1_bin_encode_neg[['mu1','mu2','mu3']], left_on='Official_name', right_index=True,suffixes=('','_neg'))

In [ ]:
Dataset1_DCP_bin = Dataset1[(Dataset1.Acceptor=='3,4-Dichlorothiophenol')&(Dataset1.Measurement_type=='Activity_score')]
Dataset1_DCP_labels = torch.tensor([[1,0] if float(val) == 1 else [0,1] for val in Dataset1_DCP_bin['Measurement_val']])
Dataset1_DCP_labels_neg = torch.tensor([[1,0] if float(val) == 0 else [0,1] for val in Dataset1_DCP_bin['Measurement_val']])
with open('../datasets/Dataset1_DCP.faa','w') as f:
    for enz,seq in zip(Dataset1_DCP_bin['Official_name'],Dataset1_DCP_bin['Sequence']):
        f.write(f'>{enz}\n{seq}\n')
dataset1_bin_encode = encode_custom_seqs_conditional_3D('Dataset1_DCP',hmm_name,run.result,Dataset1_DCP_labels,fasta_ext='faa',model= f'vae_fold_0.model')
Dataset1_DCP_bin = Dataset1_DCP_bin.merge(dataset1_bin_encode, left_on='Official_name', right_index=True)
dataset1_bin_encode_neg = encode_custom_seqs_conditional_3D('Dataset1_DCP',hmm_name,run.result,Dataset1_DCP_labels_neg,fasta_ext='faa',model= f'vae_fold_0.model')
Dataset1_DCP_bin = Dataset1_DCP_bin.merge(dataset1_bin_encode_neg[['mu1','mu2','mu3']], left_on='Official_name', right_index=True,suffixes=('','_neg'))

In [ ]:
# Projection to the latent space
# Show your queries
#run.weights = f'{run.result}/model/vae_fold_0_e9999.model'
latent_space = LatentSpace(run)
msa_embeddings = latent_space.msa_embeddings["mus"]


query_coords = latent_space.key_to_embedding(run.fixed_sequences)


conditional = True
# Prepare arrays to store points for each label
red_points = []
blue_points = []

if conditional:
    for i, label in enumerate(msa_key_label_order.values()):
        if label[0] == 0:
        #if label[0] == 1:
            red_points.append(msa_embeddings[i].numpy())
        else:
            blue_points.append(msa_embeddings[i].numpy())

# Convert to NumPy arrays for easier handling
red_points = np.array(red_points)
blue_points = np.array(blue_points)

angles = [(20, 30), (20, 80), (20, 130), (20, 180), (20, 240), (90, 0)]

# Create a figure with 4 subplots for different angles
fig = plt.figure(figsize=(24, 18))

for idx, (elev, azim) in enumerate(angles):
    ax = fig.add_subplot(2, 3, idx + 1, projection='3d')
    
    
    # Plot all red points in one scatter call
    if red_points.size > 0:
        ax.scatter(red_points[:, 0], red_points[:, 1], red_points[:, 2], alpha=0.1, s=3, color='red')

    # Plot all blue points in one scatter call
    if blue_points.size > 0:
        ax.scatter(blue_points[:, 0], blue_points[:, 1], blue_points[:, 2], alpha=0.1, s=3, color='blue')

    # Set labels and title for each subplot
    ax.set_xlabel("$Z_1$")
    ax.set_ylabel("$Z_2$")
    ax.set_zlabel("$Z_3$")
    ax.set_title(f"View (elev={elev}, azim={azim})")

    # Plot Validation Data
    ax.scatter(Dataset1_Dichloroaniline_bin.mu1, Dataset1_Dichloroaniline_bin.mu2, Dataset1_Dichloroaniline_bin.mu3, c=Dataset1_Dichloroaniline_bin.Measurement_val.to_numpy(dtype=float), alpha=1, s=30,edgecolor='black', zorder=3,cmap='coolwarm')

    # Set the view angle
    ax.view_init(elev=elev, azim=azim)

# Adjust layout
plt.tight_layout()
plt.savefig(os.path.join(run.results, f"latent_space_3d_wDCA.png"))
plt.show()

In [ ]:
# Projection to the latent space
# Show your queries
#run.weights = f'{run.result}/model/vae_fold_0_e9999.model'
latent_space = LatentSpace(run)
msa_embeddings = latent_space.msa_embeddings["mus"]


query_coords = latent_space.key_to_embedding(run.fixed_sequences)


conditional = True
# Prepare arrays to store points for each label
red_points = []
blue_points = []

if conditional:
    for i, label in enumerate(msa_key_label_order.values()):
        if label[0] == 0:
        #if label[0] == 1:
            red_points.append(msa_embeddings[i].numpy())
        else:
            blue_points.append(msa_embeddings[i].numpy())

# Convert to NumPy arrays for easier handling
red_points = np.array(red_points)
blue_points = np.array(blue_points)

angles = [(20, 30), (20, 80), (20, 130), (20, 180), (20, 240), (90, 0)]
Dataset1_Dichloroaniline_bin_subset = Dataset1_Dichloroaniline_bin[Dataset1_Dichloroaniline_bin.Measurement_val.to_numpy(dtype=float)==1]
# Create a figure with 4 subplots for different angles
fig = plt.figure(figsize=(24, 18))

for idx, (elev, azim) in enumerate(angles):
    ax = fig.add_subplot(2, 3, idx + 1, projection='3d')
    
    
    # Plot all red points in one scatter call
    if red_points.size > 0:
        ax.scatter(red_points[:, 0], red_points[:, 1], red_points[:, 2], alpha=0.1, s=3, color='red')

    # Plot all blue points in one scatter call
    if blue_points.size > 0:
        ax.scatter(blue_points[:, 0], blue_points[:, 1], blue_points[:, 2], alpha=0.1, s=3, color='blue')

    # Set labels and title for each subplot
    ax.set_xlabel("$Z_1$")
    ax.set_ylabel("$Z_2$")
    ax.set_zlabel("$Z_3$")
    ax.set_title(f"View (elev={elev}, azim={azim})")
    
    # Plot Validation Data
    ax.scatter(Dataset1_Dichloroaniline_bin_subset.mu1, Dataset1_Dichloroaniline_bin_subset.mu2, Dataset1_Dichloroaniline_bin_subset.mu3, alpha=0.8, s=30,edgecolor='black', zorder=1,color='red')
    ax.scatter(Dataset1_Dichloroaniline_bin_subset.mu1_neg, Dataset1_Dichloroaniline_bin_subset.mu2_neg, Dataset1_Dichloroaniline_bin_subset.mu3_neg, alpha=0.8, s=30,edgecolor='black', zorder=4,color='blue')

    # Set the view angle
    ax.view_init(elev=elev, azim=azim)

# Adjust layout
plt.tight_layout()
plt.savefig(os.path.join(run.results, f"latent_space_3d_wDCA_pos_reversed.png"))
plt.show()

In [ ]:
# Projection to the latent space
# Show your queries
#run.weights = f'{run.result}/model/vae_fold_0_e9999.model'
latent_space = LatentSpace(run)
msa_embeddings = latent_space.msa_embeddings["mus"]


query_coords = latent_space.key_to_embedding(run.fixed_sequences)


conditional = True
# Prepare arrays to store points for each label
red_points = []
blue_points = []

if conditional:
    for i, label in enumerate(msa_key_label_order.values()):
        if label[0] == 0:
        #if label[0] == 1:
            red_points.append(msa_embeddings[i].numpy())
        else:
            blue_points.append(msa_embeddings[i].numpy())

# Convert to NumPy arrays for easier handling
red_points = np.array(red_points)
blue_points = np.array(blue_points)

angles = [(20, 30), (20, 80), (20, 130), (20, 180), (20, 240), (90, 0)]
Dataset1_Dichloroaniline_bin_subset = Dataset1_Dichloroaniline_bin[Dataset1_Dichloroaniline_bin.Measurement_val.to_numpy(dtype=float)==0]
# Create a figure with 4 subplots for different angles
fig = plt.figure(figsize=(24, 18))

for idx, (elev, azim) in enumerate(angles):
    ax = fig.add_subplot(2, 3, idx + 1, projection='3d')
    
    
    # Plot all red points in one scatter call
    if red_points.size > 0:
        ax.scatter(red_points[:, 0], red_points[:, 1], red_points[:, 2], alpha=0.1, s=3, color='red')

    # Plot all blue points in one scatter call
    if blue_points.size > 0:
        ax.scatter(blue_points[:, 0], blue_points[:, 1], blue_points[:, 2], alpha=0.1, s=3, color='blue')

    # Set labels and title for each subplot
    ax.set_xlabel("$Z_1$")
    ax.set_ylabel("$Z_2$")
    ax.set_zlabel("$Z_3$")
    ax.set_title(f"View (elev={elev}, azim={azim})")
    
    # Plot Validation Data
    ax.scatter(Dataset1_Dichloroaniline_bin_subset.mu1, Dataset1_Dichloroaniline_bin_subset.mu2, Dataset1_Dichloroaniline_bin_subset.mu3, alpha=0.8, s=30,edgecolor='black', zorder=1,color='blue')
    ax.scatter(Dataset1_Dichloroaniline_bin_subset.mu1_neg, Dataset1_Dichloroaniline_bin_subset.mu2_neg, Dataset1_Dichloroaniline_bin_subset.mu3_neg, alpha=0.8, s=30,edgecolor='black', zorder=4,color='red')

    # Set the view angle
    ax.view_init(elev=elev, azim=azim)

# Adjust layout
plt.tight_layout()
plt.savefig(os.path.join(run.results, f"latent_space_3d_wDCA_neg_reversed.png"))
plt.show()

### Taxonomy Examination

In [ ]:
run.result

In [20]:
PFAM_UniProt_GT_filter = pd.read_csv('../../random_data/PFAM_UniProt_GT_filter.csv',index_col=0)
msa_taxonomy = []
for protein in SeqIO.parse(f'{run.result}/msa/training_msa.fasta', "fasta"):
    protein_name = protein.id
    prot_id = protein_name.split('|')[0]
    if prot_id in PFAM_UniProt_GT_filter.index:
        msa_taxonomy.append([protein_name,PFAM_UniProt_GT_filter.loc[prot_id].superkingdom,PFAM_UniProt_GT_filter.loc[prot_id].kingdom,PFAM_UniProt_GT_filter.loc[prot_id].phylum,PFAM_UniProt_GT_filter.loc[prot_id]['class'],PFAM_UniProt_GT_filter.loc[prot_id].order,PFAM_UniProt_GT_filter.loc[prot_id].family,PFAM_UniProt_GT_filter.loc[prot_id].genus,PFAM_UniProt_GT_filter.loc[prot_id]['EC number'],PFAM_UniProt_GT_filter.loc[prot_id]['Gene Ontology (GO)']])
msa_taxonomy = pd.DataFrame(msa_taxonomy,columns=['UniProt','superkingdom','kingdom','phylum','class','order','family','genus','EC','GO'])
msa_taxonomy = msa_taxonomy.set_index('UniProt')

In [ ]:
All_data_df.groupby('Acceptor').count().T['3,4-Dichlorothiophenol']

In [ ]:


msa_embeddings = latent_space.msa_embeddings["mus"]
colors_kingdom = ['lightcoral','lightblue','gold','forestgreen']
fig_lat, ax = plt.subplots(1, 1)
ax.scatter(All_data_unique.mu1, All_data_unique.mu2, color='gray', alpha=1, s=30,edgecolor='black', zorder=3)
custom_sequences = []  # give keys to MSA and embed them to the latent space
for i,kingdom in enumerate(pd.unique(msa_taxonomy.kingdom)):
    msa_filter = msa_taxonomy[msa_taxonomy.kingdom == kingdom]
    msa_filter_coords = latent_space.key_to_embedding(msa_filter.index)
    ax.scatter(msa_filter_coords[0::2], msa_filter_coords[1::2], color=colors_kingdom[i], alpha=0.5, s=5, zorder=2, label = kingdom)

#ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, )
#ax.plot(query_coords[0::2], query_coords[1::2], '.', color='red')
ax.legend(fontsize=10)
ax.set_xlabel("$Z_1$")
ax.set_ylabel("$Z_2$")
ax.set_title(f"Latent space projection, Kingdoms, Experimental Enzymes")
fig_lat.savefig(os.path.join(run.results, f"latent_space_kingdom_exp_enz.png"))

In [ ]:
msa_taxonomy

In [ ]:
for epoch in range(1,15000):
    if (epoch + 1) % 5000 == 0:
        run.weights = f'{run.result}model/vae_fold_0_e{epoch}.model'
        latent_space = LatentSpace(run)
        msa_embeddings = latent_space.msa_embeddings["mus"]
        colors_kingdom = ['lightcoral','lightblue','gold','forestgreen']
        fig_lat, ax = plt.subplots(1, 1)
        custom_sequences = []  # give keys to MSA and embed them to the latent space
        for i,kingdom in enumerate(pd.unique(msa_taxonomy.kingdom)):
            msa_filter = msa_taxonomy[msa_taxonomy.kingdom == kingdom]
            msa_filter_coords = latent_space.key_to_embedding(msa_filter.index)
            ax.scatter(msa_filter_coords[0::2], msa_filter_coords[1::2], color=colors_kingdom[i], alpha=0.5, s=5, zorder=2, label = kingdom)

        #ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, )
        #ax.plot(query_coords[0::2], query_coords[1::2], '.', color='red')
        ax.legend(fontsize=10)
        ax.set_xlabel("$Z_1$")
        ax.set_ylabel("$Z_2$")
        ax.set_title(f"Latent space projection epoch {epoch}, Kingdom")
        fig_lat.savefig(os.path.join(run.results, f"latent_space_e{epoch}_kingdom.png"))

In [ ]:
#run.weights = f'{run.results[:-7]}model/vae_fold_0_e2999.model'
latent_space = LatentSpace(run)
#msa_embeddings = latent_space.msa_embeddings["mus"]
colors_kingdom = ['lightcoral','lightblue','gold','forestgreen','lightgreen','darkorange','purple','pink','brown','gray','black','red','blue','green','orange','violet','cyan','magenta','yellow','darkred','darkblue','darkgreen','darkorange','darkviolet','darkcyan','darkmagenta','maroon','navy','olive','teal','fuchsia','silver','lime','aqua','white','lightgray','gray','black']
#fig_lat, ax = plt.subplots(1, 1)
fig_lat, axes = plt.subplots(2, 2, figsize=(12, 12),sharex=True,sharey=True)
custom_sequences = []  # give keys to MSA and embed them to the latent space
for i,kingdom in enumerate(pd.unique(msa_taxonomy.kingdom)):
    ax = axes[i//2,i%2]
    msa_kingdoms = msa_taxonomy[msa_taxonomy.kingdom == kingdom]
    for i,phylum in enumerate(pd.unique(msa_kingdoms['phylum'])):
        msa_filter = msa_kingdoms[msa_kingdoms['phylum'] == phylum]
        msa_filter_coords = latent_space.key_to_embedding(msa_filter.index)
        ax.scatter(msa_filter_coords[0::2], msa_filter_coords[1::2], color=colors_kingdom[i], alpha=0.5, s=5, zorder=2, label = phylum)

#ax.plot(msa_embeddings[:, 0], msa_embeddings[:, 1], '.', alpha=0.1, markersize=3, )
#ax.plot(query_coords[0::2], query_coords[1::2], '.', color='red')
    ax.legend(fontsize=10)
    ax.set_xlabel("$Z_1$")
    ax.set_ylabel("$Z_2$")
    ax.set_title(f'{kingdom}')
fig_lat.suptitle(f"Latent space projection final, phylum",x=0.5, y=1.02)
plt.tight_layout()
fig_lat.savefig(os.path.join(run.results, f"latent_space_kingdom_phylum.png"))

In [ ]:
msa_filter_coords = latent_space.key_to_embedding(msa_metazoa.index)
coords_z1 = msa_filter_coords[0::2]
coords_z2 = msa_filter_coords[1::2]
mask_ = ((coords_z1<4) & (coords_z2>5))
for id in msa_metazoa.iloc[mask_,:].index.str.split('|').str[0]:
    print(id)


### Decoding

In [159]:
vae = latent_space.vae
z = torch.tensor(Dataset1_DCP_bin[['mu1','mu2']].to_numpy())
c = Dataset1_DCP_labels

#if c is not None:
#    c = c.repeat(z.shape[0], 1)  # Repeat `c` for each row in `z`, but `c` already matches the batch size here

seq_decoding = {}
for j in range(10):
    with torch.no_grad():
        number_sequences = vae.z_to_number_sequences(z[0:2], c[0:2]).cpu().numpy()
        test_dict_num = {f"test{i}": seq for i, seq in enumerate(number_sequences)}
        test_to_store = MSA.number_to_amino(test_dict_num)
        seq_decoding[j] = test_to_store['test0']
seq_decoding_neg = {}
c = Dataset1_DCP_labels_neg
for j in range(10):
    with torch.no_grad():
        number_sequences = vae.z_to_number_sequences(z[0:2], c[0:2]).cpu().numpy()
        test_dict_num = {f"test{i}": seq for i, seq in enumerate(number_sequences)}
        test_to_store = MSA.number_to_amino(test_dict_num)
        seq_decoding_neg[j] = test_to_store['test0']

In [ ]:
for i in range(1):
    print(''.join(seq_decoding[i]))
    print(''.join(seq_decoding_neg[i]))